In [1]:
# %autoreload 2

import sys
sys.path.append('../data_loader')
sys.path.append('../textgen')
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from pprint import pprint as print


In [2]:
import re
import neologdn

def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

In [3]:
import novel
import config
from torch.utils.data import Dataset

print(config.NOVEL_DATA_PATH)

data_loader = novel.DataLoader(novel.Dataset(config.NOVEL_DATA_PATH))

for i,body in enumerate(map(lambda l: preprocess_text(l), data_loader)):
    if i > 3:
        break
    print(body[0:100])

PosixPath('/workspaces/pytorch-practice/data/novels/narou')
'勇者と魔王の戦い。それは、この世界で幾度となく繰り返されてきたものだ。魔族を統べる魔王。彼らは一つの時代に必ず一人いる。今代の魔王が倒された時、魔族や魔物の中でも特に強い力を持ったものが、次代の魔王と'
'さて、私が蜘蛛に転生してしまったことは、とても遺憾ながら認めよう。認めはしたけど、このあとどうしよう?ボリボリッ!何やら不穏な音がする。うん。現実から目をそらせちゃ、ダメだ。私の目の前には私のおそらく'
'吾輩は蜘蛛である。名前はまだない。え、突然何言ってんだって?私って名前ないらしいから、それを言ってみただけ。何の話かって?それを話すにはちょっと前のことを振り返らなきゃならない。***********'
'さてさて。いつまでも同じ場所にいても仕方ないし、そろそろ移動しないとまずいよね。足跡があるってことは、人がいるってことだし、そいつらと鉢合わせになったら面倒なことになる予感しかしない。とりあえず、足跡'


In [4]:
from pathlib import Path

prefix_len = len(str(config.NOVEL_DATA_PATH))
for i,p in enumerate(config.NOVEL_DATA_PATH.glob('*/N*/*.txt')):
    if i >= 10:
        break
    print(str(p)[prefix_len+1:])

episode_paths = [str(ep)[prefix_len+1:] for ep in config.NOVEL_DATA_PATH.glob('*/N*/*.txt')]

'0/N7975CR/N7975CR-1.txt'
'0/N7975CR/N7975CR-10.txt'
'0/N7975CR/N7975CR-100.txt'
'0/N7975CR/N7975CR-101.txt'
'0/N7975CR/N7975CR-102.txt'
'0/N7975CR/N7975CR-103.txt'
'0/N7975CR/N7975CR-104.txt'
'0/N7975CR/N7975CR-105.txt'
'0/N7975CR/N7975CR-106.txt'
'0/N7975CR/N7975CR-107.txt'


In [5]:
from sklearn.model_selection import train_test_split

train_ep, test_ep = train_test_split(episode_paths, test_size=0.2, random_state=42, shuffle=True)

len(train_ep), len(test_ep)

(178220, 44556)

In [8]:
with Path(config.SENTENCEPIECE_MODEL_DIR / 'clean.txt').open(mode='w') as fo:
    for i,ep in enumerate(episode_paths):
        with Path(config.NOVEL_DATA_PATH / ep).open() as fi:
            clean_text = preprocess_text(fi.read())
            fo.write(clean_text + "\n")


In [9]:
import sentencepiece as spm

spm.SentencePieceTrainer.Train(
    f'--input={str(config.SENTENCEPIECE_MODEL_DIR)}/clean.txt' \
    f' --model_prefix={str(config.SENTENCEPIECE_MODEL_DIR)}/novel_sp' \
    ' --character_coverage=0.9995' \
    ' --vocab_size=52000' \
    ' --pad_id=3' \
    ' --add_dummy_prefix=False'
)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/workspaces/pytorch-practice/notes/../models/sentencepiece/clean.txt --model_prefix=/workspaces/pytorch-practice/notes/../models/sentencepiece/novel_sp --character_coverage=0.9995 --vocab_size=52000 --pad_id=3 --add_dummy_prefix=False
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /workspaces/pytorch-practice/notes/../models/sentencepiece/clean.txt
  input_format: 
  model_prefix: /workspaces/pytorch-practice/notes/../models/sentencepiece/novel_sp
  model_type: UNIGRAM
  vocab_size: 52000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  requ

In [16]:
sp = spm.SentencePieceProcessor()
sp.Load(f'{str(config.SENTENCEPIECE_MODEL_DIR)}/novel_sp.model')
print(sp.EncodeAsPieces('はじめての自然言語処理'))
print(sp.EncodeAsPieces('勇者はそんな魔王と戦う、人族の希望。'))
print(sp.EncodeAsPieces('〈黒穴〉に飛び込んだ者は、莫大な財宝や、強大な能力を得るという。'), width=10000)

['はじめて', 'の', '自然', '言語', '処理']
['勇者は', 'そんな', '魔王', 'と戦う', '、', '人族の', '希望', '。']
['〈', '黒', '穴', '〉', 'に', '飛び込んだ', '者は', '、', '莫大な', '財宝', 'や', '、', '強大な', '能力', 'を得る', 'という', '。']
